In [ ]:
import re
!python -m pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Entity:
  def __init__(self, var_name=None,desc=None):
    self.var_name = var_name
    self.desc = desc

  def show_params(self):
    print('var_name',self.var_name)
    print('desc',self.desc)

class Cross(Entity):
  def __init__(self,weight, var_name=None,desc=None):
    self.weight = weight
    self.var_name = var_name
    self.desc = desc

  def show_params(self):
    print('var_name',self.var_name)
    print('weight',self.weight)
    print('desc',self.desc)

class Entity_Nameable(Entity):
  def __init__(self,name, var_name=None,desc=None):
    self.var_name = var_name
    self.name = name
    self.desc = desc
    
  def show_params(self):
    print('var_name',self.var_name)
    print('name',self.name)
    print('desc',self.desc)

class Entity_Trajectorial(Entity_Nameable):
  def __init__(self,name, var_name=None,desc=None,trj=None):
    self.var_name = var_name
    self.name = name
    self.desc = desc
    self.trj = trj
  def show_params(self):
    print('var_name',self.var_name)
    print('name',self.name)
    print('desc',self.desc)
    print('trj',self.trj)

In [ ]:
def del_comms(text):
  text = re.sub('<<.*?>>','',text,flags=re.DOTALL)
  text = re.sub('//.*','',text)
  return text

In [ ]:
class Serializer:

  def serialize_var_name(self,text,entity):
    v = re.findall(entity+r' ?[^{]+?[\{=]{1}',text)
    if (len(v)!=0):
      var_name = v[0].split()[1]
      var_name = var_name if var_name!='{' and var_name!='=' else None
    else:
      var_name = None
    return var_name
  
  def serialize_name(self,text,entity,var_name):
    n1 = re.findall(r'Name *?= *?".+?"',text)
    n2 = re.findall(entity+r' *?'+var_name+' *?= *?".+?"',text) if var_name is not None else []
    try:
      if (len(n2)!=0):
        name = n2[0].split('"')[1]
      else:
        name = n1[0].split('"')[1]

      return name

    except:
      print(f'Please type the name of the object in {text}')

  def serialize_desc(self,text):
    d = re.findall(r'Desc *?= *?".+?"',text)
    desc = d[0].split('"')[1] if (len(d)!=0) else None
    return desc

  def serialize_trajectory(self,text):
    t = re.findall(r'Trj *?= *?[0-9]{1}',text)
    if (len(t)==0):
      return None
    trj = ''.join(t[0].split()).split('=')[1]
    return trj
  
  def serialize_weight(self,text):
    w = re.findall(r'Weight *?= *?"x{0,3}"',text)
    weight = w[0].split('"')[1] if (len(w)!=0) else None
    return weight

  def serialize(self,text,entity):
    var_name = self.serialize_var_name(text,entity)
    desc = self.serialize_desc(text)
    
    if (entity == 'EmpNeed' or entity == 'Technology'):
      name = self.serialize_name(text,entity,var_name)
      ent = Entity_Nameable(name,var_name,desc)
    elif (entity == 'LrnOut' or entity == 'Course'):
      name = self.serialize_name(text,entity,var_name)
      trj = self.serialize_trajectory(text)
      ent = Entity_Trajectorial(name,var_name,desc,trj)
    elif (entity == 'Cross'):
      weight = self.serialize_weight(text)
      ent = Cross(weight,var_name,desc)
    return ent

In [ ]:
class Extractor:

  def __init__(self,text):
    self.text = text
    
    self.empNeeds = []
    self.techs = []
    self.outcomes = []
    self.courses = []

    self.serializer = Serializer()

  def extract_matrix(self):
    self.xm = re.findall(r'Xmat.+',self.text)[0].split()[1]

  def extract_operations(self):

    empOper = self.extract_entity_assignments(self.text,'EmpNeeds',self.xm)
    empOper.extend(self.extract_entity_actions(self.text,'EmpNeeds',self.xm))
    empOper.sort(key=lambda x: x.start())

    techOper = self.extract_entity_assignments(self.text,'Technologies',self.xm)
    techOper.extend(self.extract_entity_actions(self.text,'Technologies',self.xm))
    techOper.sort(key=lambda x: x.start())

    outOper = self.extract_entity_assignments(self.text,'Outcomes',self.xm)
    outOper.extend(self.extract_entity_actions(self.text,'Outcomes',self.xm))
    outOper.sort(key=lambda x: x.start())

    crsOper = self.extract_entity_assignments(self.text,'Courses',self.xm)
    crsOper.extend(self.extract_entity_actions(self.text,'Courses',self.xm))
    crsOper.sort(key=lambda x: x.start())

    ETOper = self.extract_entity_assignments(self.text,'CrossEmpTechs',self.xm)
    ETOper.extend(self.extract_entity_actions(self.text,'CrossEmpTechs',self.xm))
    ETOper.sort(key=lambda x: x.start())

    ELOper = self.extract_entity_assignments(self.text,'CrossEmpLearns',self.xm)
    ELOper.extend(self.extract_entity_actions(self.text,'CrossEmpLearns',self.xm))
    ELOper.sort(key=lambda x: x.start())

    CTOper = self.extract_entity_assignments(self.text,'CrossCourseTechs',self.xm)
    CTOper.extend(self.extract_entity_actions(self.text,'CrossCourseTechs',self.xm))
    CTOper.sort(key=lambda x: x.start())

    CLOper = self.extract_entity_assignments(self.text,'CrossCourseLearns',self.xm)
    CLOper.extend(self.extract_entity_actions(self.text,'CrossCourseLearns',self.xm))
    CLOper.sort(key=lambda x: x.start())

    return empOper,techOper,outOper,crsOper,ETOper,ELOper,CTOper,CLOper

  def extract_ents(self):

    self.all_needs = self.extract_entities(self.text,'EmpNeed')
    self.all_needs = list(map(lambda x: self.serializer.serialize(x[0],'EmpNeed'),self.all_needs))

    self.all_techs = self.extract_entities(self.text,'Technology')
    self.all_techs = list(map(lambda x: self.serializer.serialize(x[0],'Technology'),self.all_techs))

    self.all_outs = self.extract_entities(self.text,'LrnOut')
    self.all_outs = list(map(lambda x: self.serializer.serialize(x[0],'LrnOut'),self.all_outs))

    self.all_courses = self.extract_entities(self.text,'Course')
    self.all_courses = list(map(lambda x: self.serializer.serialize(x[0],'Course'),self.all_courses))

    self.all_crosses = self.extract_entities(self.text,'Cross')
    self.all_crosses = list(map(lambda x: self.serializer.serialize(x[0],'Cross'),self.all_crosses))
  
  def form_list(self):
    self.extract_matrix()
    self.extract_ents()
    empOper,techOper,outOper,crsOper,ETOper,ELOper,CTOper,CLOper = self.extract_operations()

    self.process_operations(empOper,'EmpNeed',self.empNeeds,self.all_needs)
    self.process_operations(techOper,'Technology',self.techs,self.all_techs)
    self.process_operations(outOper,'LrnOut',self.outcomes,self.all_outs)
    self.process_operations(crsOper,'Course',self.courses,self.all_courses)

    self.crossEmpTechs = [[ None for i in range(len(self.techs)) ] for j in range(len(self.empNeeds))]
    self.crossEmpLearns = [[ None for i in range(len(self.outcomes)) ] for j in range(len(self.empNeeds))]
    self.crossCourseTechs = [[ None for i in range(len(self.techs)) ] for j in range(len(self.courses))]
    self.crossCourseLearns = [[ None for i in range(len(self.outcomes)) ] for j in range(len(self.courses))]

    self.process_cross_operations(ETOper,self.crossEmpTechs,self.empNeeds,self.techs)
    self.process_cross_operations(ELOper,self.crossEmpLearns,self.empNeeds,self.outcomes)
    self.process_cross_operations(CTOper,self.crossCourseTechs,self.courses,self.techs)
    self.process_cross_operations(CLOper,self.crossCourseLearns,self.courses,self.outcomes)

  def process_operations(self,opers,entity,entities,all_entities):
    for oper in opers:
      if ('.Add' in oper[0]):
        ent = self.add_process(oper[0],all_entities)
        entities.append(ent)
      elif('.RemoveAt' in oper[0]):
        index = self.removeAt_process(oper[0])
        entities.pop(index)
      elif('.Remove' in oper[0]):
        ent_to_delete = self.remove_process(oper[0],entities)
        entities.remove(ent_to_delete)
      elif('=' in oper[0]):
        entities.clear()
        ents = self.assignment_process(oper[0],entity)
        for ent in ents:
          entities.append(self.serializer.serialize(ent[0],entity))
        if len(ents)==0:
          var_name = ''.join(oper[0].split()).split('=')[1]
          for ent in all_entities:
            if (ent.var_name==var_name):
              entities.append(ent)
  
  def process_cross_operations(self,opers,entities,feature1,feature2):
    for oper in opers:
      i,j = self.cross_process(oper[0],feature1,feature2)
      if('.Remove' in oper[0]):
        entities[i][j]=None
      else:
        entities[i][j] = self.serializer.serialize(oper[0],"Cross")
        var_name = ''.join(oper[0].split()).split('=')[1]
        if ("{" not in var_name):
          for cross in self.all_crosses:
            if (cross.var_name == var_name):
              entities[i][j] = cross
              break

  def add_process(self,text,all_entities):
    x = re.split('\(|\)',text)[1]
    for ent in all_entities:
      if (ent.var_name==x):
        return ent
  
  def remove_process(self,text,entities):
    x = re.split('\(|\)',text)[1]
    for ent in entities:
      if (ent.var_name==x):
        return ent
  
  def cross_process(self,text,f1,f2):
    cross_coordinates = re.split('\[|\]',text)
    if re.fullmatch(r'\d+',cross_coordinates[1]) is not None:
      first = int(cross_coordinates[1])
    else:
      for i,ent in enumerate(f1):
        if (ent.var_name==cross_coordinates[1]):
          first=i
    if re.fullmatch(r'\d+',cross_coordinates[3]) is not None:
      second = int(cross_coordinates[3])
    else:
      for j,ent in enumerate(f2):
        if (ent.var_name==cross_coordinates[3]):
          second=j
    return first,second

  def removeAt_process(self,text):
    x = re.split('\(|\)',text)[1]
    return int(x)
  
  def assignment_process(self,text,entity):
    ents = self.extract_entities(text,entity)
    return ents
  
  def extract_entities(self,text,entity):
    entities = []
    a = re.finditer(entity+r' [^{]+= ?".*?"',text,flags=re.DOTALL)
    b = re.finditer(entity+r' [^{"\']+?= ?{.+?}',text,flags=re.DOTALL)
    c = re.finditer(entity+r' +?{.+?}',text,flags=re.DOTALL)
    entities.extend(a)
    entities.extend(b)
    entities.extend(c)
    return entities

  def extract_entity_assignments(self,text,entity,matrixName):
    signs = []
    a = re.finditer(matrixName+'\.'+entity+r' ?= ?\[.+?\]',text,flags=re.DOTALL) # xm.EmpNeeds = [EmpNeed {Name = "name"}}]
    b = re.finditer(matrixName+'\.'+entity+r' ?=[^\[\n ]+',text) # xm.EmpNeeds = need
    if ('Cross' in entity):
      c = re.finditer(matrixName+'\.'+entity+r'\[.+?\]\[.+?\] *?= *?.+',text) # xm.CrossEmpTechs[need][big_data] = Cross {Weight="x", Desc=None}
      signs.extend(c)
    signs.extend(a)
    signs.extend(b)
    return signs

  def extract_entity_actions(self,text,entity,matrixName):
    if ('Cross' not in entity):
      return list(re.finditer(matrixName+'\.'+entity+r'\..+?\(.+?\)',text))
    return list(re.finditer(matrixName+'\.'+entity+r'\[.+?\]\[.+?\]\..+?\(.*?\)',text))

In [ ]:
class Cross_Matrix_Validator:

  def __init__(self,empNeeds,techs,outcomes,courses,ET,EL,CT,CL):
    self.empNeeds = empNeeds
    self.techs = techs
    self.outcomes = outcomes
    self.courses = courses
    self.ET = ET
    self.EL = EL
    self.CT = CT
    self.CL = CL
  
  def tech_validate(self):
    not_valid = []
    for tech in self.techs:
      max = ''
      index = self.find_by_name(tech,self.techs)
      for cross in self.ET:
        if (cross[index] is not None):
          if (cross[index].weight>max):
            max=cross[index].weight
      max_ET = max
      max = ''
      for cross in self.CT:
        if (cross[index] is not None):
          if (cross[index].weight>max):
            max=cross[index].weight
      max_CT = max
      if (max_CT>max_ET):
        not_valid.append(index)
    return not_valid

  def course_validate(self):
    not_valid = []
    for course in self.courses:
      crs_index = self.find_by_name(course,self.courses)
      for out in self.outcomes:
        out_index = self.find_by_name(out,self.outcomes)
        for tech in self.techs:
          tech_index = self.find_by_name(tech,self.techs)
          if (self.CT[crs_index][tech_index] is not None and self.CL[crs_index][out_index] is not None):
            if (self.CT[crs_index][tech_index].weight<self.CL[crs_index][out_index].weight):
              not_valid.append((crs_index,tech_index,out_index))
          elif (self.CT[crs_index][tech_index] is None and self.CL[crs_index][out_index] is not None):
            if (''<self.CL[crs_index][out_index].weight):
              not_valid.append((crs_index,tech_index,out_index))
    return not_valid


  def find_by_name(self,entity,entities):
    for i,ent in enumerate(entities):
      if (ent.name==entity.name):
        return i

In [ ]:
with open("X_matrix_BDaML.txt", encoding = 'utf-8') as f:
   x = f.read()
x = del_comms(x)

In [ ]:
ext = Extractor(x)

In [ ]:
ext.form_list()

In [ ]:
val = Cross_Matrix_Validator(ext.empNeeds,ext.techs,ext.outcomes,ext.courses,ext.crossEmpTechs,ext.crossEmpLearns,ext.crossCourseTechs,ext.crossCourseLearns)

In [ ]:
n_needs = len(ext.empNeeds)
n_techs = len(ext.techs)
n_outs = len(ext.outcomes)
n_courses = len(ext.courses)
h = n_needs+n_courses+1
w = n_techs+n_outs+1

In [ ]:
need_names = [need.name for need in ext.empNeeds]
tech_names = [tech.name for tech in ext.techs]
out_names = [out.name for out in ext.outcomes]
course_names = [course.name for course in ext.courses]

In [ ]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
align=Alignment(horizontal='center',
                    vertical='center',
                     wrap_text=False,
                     shrink_to_fit=True)
rotation=Alignment(horizontal='center',
                    vertical='center',
                     text_rotation=90,
                     wrap_text=False,
                     shrink_to_fit=False)
border = Border(left=Side(border_style='medium',
                            color='FF000000'),
                 right=Side(border_style='medium',
                            color='FF000000'),
                 top=Side(border_style='medium',
                          color='000000'),
                 bottom=Side(border_style='medium',
                             color='FF000000'),
                )
border_cross = Border(left=Side(border_style='medium',
                            color='FF000000'),
                 right=Side(border_style='medium',
                            color='FF000000'),
                 top=Side(border_style='medium',
                          color='000000'),
                 bottom=Side(border_style='medium',
                             color='FF000000'),
                 diagonal=Side(border_style='medium',
                               color='FF000000'),
                 diagonal_direction=0,
                 diagonalUp=Side(border_style='medium',
                               color='FF000000'),
                 diagonalDown=Side(border_style='medium',
                               color='FF000000')
                )

In [ ]:
from openpyxl import Workbook
from openpyxl.comments import Comment

def create_workbook(path,workbook=None):
    if workbook is None:
      workbook = Workbook()
    sheet = workbook.active
    start_point = "C3"

    # чёрные границы
    for col in range(n_techs+n_outs+1):
      for row in range(n_needs+n_courses+1):
        sheet[f'{chr(ord(start_point[0])+col)}{int(start_point[1])+row}'].border = border

    #вставка сущностей
    for i,name in enumerate(need_names):
      sheet[f'{chr(ord(start_point[0])+n_techs)}{int(start_point[1])+i}'] = need_names[n_needs-i-1]
      sheet[f'{chr(ord(start_point[0])+n_techs)}{int(start_point[1])+i}'].alignment = align
    for i,name in enumerate(tech_names):
      sheet[f'{chr(ord(start_point[0])+i)}{int(start_point[1])+n_needs}'] = tech_names[n_techs-i-1]
      sheet[f'{chr(ord(start_point[0])+i)}{int(start_point[1])+n_needs}'].alignment = rotation
    for i,name in enumerate(out_names):
      sheet[f'{chr(ord(start_point[0])+n_techs+1+i)}{int(start_point[1])+n_needs}'] = out_names[i]
      sheet[f'{chr(ord(start_point[0])+n_techs+1+i)}{int(start_point[1])+n_needs}'].alignment = rotation
    for i,name in enumerate(course_names):
      sheet[f'{chr(ord(start_point[0])+n_techs)}{int(start_point[1])+n_needs+1+i}'] = course_names[i]
      sheet[f'{chr(ord(start_point[0])+n_techs)}{int(start_point[1])+n_needs+1+i}'].alignment = align
    
    #вставка креста посередине
    sheet[f'{chr(ord(start_point[0])+n_techs)}{int(start_point[1])+n_needs}'].border = border_cross

    #вставка крестиков
    for j,tech in enumerate(ext.crossEmpTechs):
      for i,emp in enumerate(tech):
        if (emp is not None):
          sheet[f'{chr(ord(start_point[0])+n_techs-i-1)}{int(start_point[1])+n_needs-j-1}'] = emp.weight
          sheet[f'{chr(ord(start_point[0])+n_techs-i-1)}{int(start_point[1])+n_needs-j-1}'].alignment = align

    for j,out in enumerate(ext.crossEmpLearns):
      for i,emp in enumerate(out):
        if (emp is not None):
          sheet[f'{chr(ord(start_point[0])+n_techs+i+1)}{int(start_point[1])+n_needs-j-1}'] = emp.weight
          sheet[f'{chr(ord(start_point[0])+n_techs+i+1)}{int(start_point[1])+n_needs-j-1}'].alignment = align

    for j,tech in enumerate(ext.crossCourseTechs):
      for i,crs in enumerate(tech):
        if (crs is not None):
          sheet[f'{chr(ord(start_point[0])+n_techs-i-1)}{int(start_point[1])+n_needs+j+1}'] = crs.weight
          sheet[f'{chr(ord(start_point[0])+n_techs-i-1)}{int(start_point[1])+n_needs+j+1}'].alignment = align

    for j,out in enumerate(ext.crossCourseLearns):
      for i,crs in enumerate(out):
        if (crs is not None):
          sheet[f'{chr(ord(start_point[0])+n_techs+i+1)}{int(start_point[1])+n_needs+j+1}'] = crs.weight
          sheet[f'{chr(ord(start_point[0])+n_techs+i+1)}{int(start_point[1])+n_needs+j+1}'].alignment = align

    #валидация крестиков
    not_valid_techs = val.tech_validate()
    for tech in not_valid_techs:
      sheet[f'{chr(ord(start_point[0])+n_techs-tech-1)}{int(start_point[1])+n_needs}'].fill = PatternFill("solid", fgColor="ff5b40")

    not_valid_courses = val.course_validate()
    for cross in not_valid_courses:
      sheet[f'{chr(ord(start_point[0])+n_techs-cross[1]-1)}{int(start_point[1])+n_needs+cross[0]+1}'].fill = PatternFill("solid", fgColor="00a4bf") # CT filling
      if (sheet[f'{chr(ord(start_point[0])+n_techs-cross[1]-1)}{int(start_point[1])+n_needs+cross[0]+1}'].comment is None):
        sheet[f'{chr(ord(start_point[0])+n_techs-cross[1]-1)}{int(start_point[1])+n_needs+cross[0]+1}'].comment = Comment(f'{chr(ord(start_point[0])+n_techs+cross[2]+1)}{int(start_point[1])+n_needs+cross[0]+1}', 'Validator')
      else:
        sheet[f'{chr(ord(start_point[0])+n_techs-cross[1]-1)}{int(start_point[1])+n_needs+cross[0]+1}'].comment = Comment(sheet[f'{chr(ord(start_point[0])+n_techs-cross[1]-1)}{int(start_point[1])+n_needs+cross[0]+1}'].comment.text+' '+f'{chr(ord(start_point[0])+n_techs+cross[2]+1)}{int(start_point[1])+n_needs+cross[0]+1}', 'Validator')
      sheet[f'{chr(ord(start_point[0])+n_techs+cross[2]+1)}{int(start_point[1])+n_needs+cross[0]+1}'].fill = PatternFill("solid", fgColor="00a4bf") #CL filling

    #подгон широты колонки под текст
    dims = {}
    for row in sheet.rows:
      for cell in row:
        if cell.value and cell.alignment!=rotation and cell.value not in ['x','xx','xxx','']:
             dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))   
    for col, value in dims.items():
      sheet.column_dimensions[col].width = value

    workbook.save(path)

In [ ]:
create_workbook('bdaml.xlsx')